# 🎫 GeniusXP

## 💡 Explicação do projeto
Uma plataforma centralizada para gestão de eventos que simplifica operações como inscrições, pagamentos e check-in, enquanto aumenta o engajamento com enquetes e networking. A inteligência artificial da GeniusXP utiliza as preferências dos usuários para oferecer uma experiência altamente personalizada e otimizar o planejamento. Com análise de sentimento e assistência virtual, a plataforma proporciona interações mais significativas, elevando a eficiência da gestão e tornando os eventos mais impactantes para cada participante.

## 📖 Metodologia utilizada
A aplicação Bluetrip utiliza técnicas avançadas de inteligência artificial para personalizar a experiência do usuário e automatizar processos, integrando o framework TensorFlow com a biblioteca FastAPI para criar uma API robusta e eficiente. A seguir, descrevemos a metodologia empregada para desenvolver este componente da aplicação.

## 👥 Equipe
Este projeto está sendo desenvolvido pelos seguintes membros:

- RM99565 - Erick Nathan Capito Pereira (2TDSPV)
- RM99577 - Guilherme Dias Gomes (2TDSPX)
- RM550841 - Lucas Araujo Oliveira Silva (2TDSPV)
- RM99409 - Michael José Bernardi Da Silva (2TDSPX)

## 🎲 Datasets utilizados

- https://www.kaggle.com/datasets/pashupatigupta/emotion-detection-from-text
- https://www.kaggle.com/datasets/praveengovi/emotions-dataset-for-nlp
- https://www.kaggle.com/datasets/simaanjali/emotion-analysis-based-on-text

## Configuração do Ambiente e Importações
Instalação das bibliotecas necessárias para o projeto, incluindo bibliotecas de machine learning, tradução, processamento de linguagem natural e conexão com banco de dados Oracle.

In [ ]:
%pip install -U scikit-learn scipy deep-translator nltk oracledb

Importações de bibliotecas para manipulação de dados, machine learning, processamento de linguagem natural, tradução, conexão com o banco de dados Oracle, e desenvolvimento da API com FastAPI.

In [19]:
import pandas as pd
import re, string
import os
from datetime import datetime

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from deep_translator import GoogleTranslator
import oracledb

from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel

import uvicorn

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\erick\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\erick\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\erick\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\erick\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\erick\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\erick\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_pe

True

# Transformação do texto
Configuração do CountVectorizer, que irá transformar o texto em vetores numéricos de acordo com os parâmetros e pré-processamento do texto com Lemmatization.

In [20]:
wl = WordNetLemmatizer()
vectorizer = CountVectorizer(lowercase=True, 
                             stop_words=stopwords.words('english'),
                             max_features=1000)

def preprocess(text):
    text = re.sub(r'@[A-Za-z0-9]+', ' ', text)
    text = text.lower() 
    text = text.strip()  
    text = re.compile('<.*?>').sub('', text) 
    text = re.compile('[%s]' % re.escape(string.punctuation)).sub(' ', text)  
    text = re.sub('\s+', ' ', text)  
    text = re.sub(r'\[[0-9]*\]',' ',text) 
    text = re.sub(r'[^\w\s]', '', str(text).lower().strip())
    text = re.sub(r'\d',' ',text) 
    text = re.sub(r'\s+',' ',text) 
    return text

def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

def lemmatizer(string):
    word_pos_tags = nltk.pos_tag(word_tokenize(string))
    a=[wl.lemmatize(tag[0], get_wordnet_pos(tag[1])) for idx, tag in enumerate(word_pos_tags)]
    return " ".join(a)

## Carregando o dataset
Carregamento do dataset 'tweet_emotions_clean.csv' em um DataFrame do pandas para análise e uso posterior. Caso não exista o dataset com os dados limpos, ele cria um novo.

In [21]:
df = None
if os.path.exists('./model/tweet_emotions_clean.csv'):
    print("Loading preprocessed data...")
    df = pd.read_csv('./model/tweet_emotions_clean.csv')
else:
    print('Preprocessing data...')
    df = pd.read_csv('./model/tweet_emotions.csv')
    
    df.drop_duplicates(inplace=True)
    df.drop_duplicates(subset='content', inplace=True)

    df['clean_content'] = df['content'].apply(lambda x: lemmatizer(preprocess(x)))
    df.to_csv('./model/tweet_emotions_clean.csv', index=False)

df

Loading preprocessed data...


,sentiment,content,clean_content
0,empty,@tiffanylue i know i was listening to bad hab...,i know i be listen to bad habit earlier and i ...
1,sadness,Layin n bed with a headache ughhhh...waiting ...,layin n bed with a headache ughhhh wait on you...
2,sadness,Funeral ceremony...gloomy friday...,funeral ceremony gloomy friday
3,enthusiasm,wants to hang out with friends SOON!,want to hang out with friend soon
4,neutral,@dannycastillo We want to trade with someone w...,we want to trade with someone who have houston...
...,...,...,...
433644,neutral,i feel that becuase of the tyranical nature of...,i feel that becuase of the tyranical nature of...
433645,neutral,i think that after i had spent some time inves...,i think that after i have spend some time inve...
433646,anger,neutral at doctors for their treatment of me a...,neutral at doctor for their treatment of me af...
433647,anger,neutral with boyfriends best friend for aliena...,neutral with boyfriend best friend for alienat...


## Separação dos dados de treino
Transformação da coluna 'content' do dataset em uma matriz esparsa de contagem de palavras, que será usada como entrada para o modelo.

In [25]:
X = vectorizer.fit_transform(df['clean_content'].values.astype('U'))

Definição da variável dependente (target) como a coluna 'sentiment', que contém os rótulos das emoções.

In [26]:
y = df['sentiment']
y.unique()

array(['empty', 'sadness', 'enthusiasm', 'neutral', 'worry', 'surprise',
       'love', 'fun', 'hate', 'happiness', 'boredom', 'relief', 'anger'],
      dtype=object)

## Separando conjuntos de treino e teste
Divisão do dataset em conjuntos de treino e teste. O conjunto de treino será usado para treinar o modelo, e o de teste para avaliação.

In [27]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,random_state=42)
X_train

<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 2013560 stored elements and shape (303554, 1000)>

## Treinando o modelo
Instanciação e treinamento do modelo de regressão logística com o conjunto de treino.

In [28]:
model = LogisticRegression(max_iter=1000, solver='sag')
model.fit(X_train, y_train)

LogisticRegression(max_iter=1000, solver='sag')

Calculando a acurácia do modelo.

In [29]:
accuracy = 100 * (accuracy_score(y_test, model.predict(X_test)))
print(f'A acurácia do modelo é de {accuracy: .2f}%')

A acurácia do modelo é de  84.67%


## Realizando a predição do sentimento baseado em uma mensagem
Função para realizar a predição da emoção de uma mensagem. A mensagem é traduzida para o inglês e então transformada em um vetor antes da predição.

In [30]:
def predict_message(message: str):
    translated_message = GoogleTranslator(source='auto', target='en').translate(text=message)
    clean_message = lemmatizer(preprocess(translated_message))
    prediction = model.predict(vectorizer.transform([clean_message]))
    return prediction[0]

## Instanciando o Banco de dados
Conexão com o banco de dados Oracle e criação de um cursor para executar operações SQL.

In [31]:
connection = oracledb.connect(
    user="rm99565",
    password="140805",
    dsn="oracle.fiap.com.br/orcl")
cursor = connection.cursor()

## Instanciando o FastAPI
Instanciação do FastAPI e configuração do middleware CORS para permitir comunicações com o front-end.

In [32]:
app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

## Criação das rotas para conexão com o front-end

### Rota de status para verificar se a aplicação está sendo executada
#### [GET] /status

In [33]:
@app.get("/status")
async def app_status():
    return 'ok'

### Rota para predição do sentimento da mensagem enviada na requisição
#### [POST] /event/{event_id}/send-message - message: str

In [34]:
@app.post("/event/{event_id}/send-message")
async def send_message(message: str, event_id: int):
    emotion = predict_message(message)

    cursor.callproc("INCREASE_EVENT_EMOTION", [event_id, emotion])
    connection.commit()
    return emotion
    

### Rota para obter sentimentos de determinado evento
#### [GET] /event/{event_id}

In [35]:

class EventDay(BaseModel):
    date: datetime
    empty: int
    sadness: int
    enthusiasm: int
    neutral: int
    worry: int
    surprise: int
    love: int
    fun: int
    hate: int
    happiness: int
    boredom: int
    relief: int
    anger: int

@app.get("/event/{event_id}")
async def get_event(event_id: int):
    cursor.execute("SELECT * FROM TBL_EVENT_EMOTIONS WHERE EVENT_ID = :id", id=event_id)
    event_days = cursor.fetchall()

    event_day_list = [EventDay(
        date=row[2],
        empty=row[3],
        sadness=row[4],
        enthusiasm=row[5],
        neutral=row[6],
        worry=row[7],
        surprise=row[8],
        love=row[9],
        fun=row[10],
        hate=row[11],
        happiness=row[12],
        boredom=row[13],
        relief=row[14],
        anger=row[15]
    ) for row in event_days]

    return event_day_list

## Execução do servidor HTTP do FastAPI utilizando Uvicorn

In [ ]:
if __name__ == "__main__":
    uvicorn.run("app:app", host="127.0.0.1", port=8000)